In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch_geometric

In [3]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html


In [4]:
cd /content/drive/MyDrive/Me\ --\ Education/Topics/AI/Machine Learning Dr. Montazeri/Project/ml_mda

/content/drive/MyDrive/Me -- Education/Topics/AI/Machine Learning Dr. Montazeri/Project/ml_mda


# Requirements

In [5]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(stream=sys.stdout)
    ],
    force=True
)

In [6]:
logger = logging.getLogger(__name__)

In [7]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
from src.optimization import DiseaseAssessClassifierTrainer, DiseaseAssessClassifierTester
from src.models import D3Classifier, D3ClassifierFactory
from src.data import MicrobeDiseaseAssociationData, MicrobeDiseaseAssociationTrainTestSpliter
from src.features import get_associations, get_entities
from src.utils import train_test_sampler
from base import cross_validation, OptimizerConfig


2024-02-22 18:30:22,976 [INFO] NumExpr defaulting to 2 threads.


# Classification

## Data

In [9]:
associations = get_associations()

train_indices, test_indices = train_test_sampler(associations.shape[0], 0.7)

data = MicrobeDiseaseAssociationData(associations)

train_data = MicrobeDiseaseAssociationData(associations.iloc[train_indices])
test_data = MicrobeDiseaseAssociationData(associations.iloc[test_indices])

## Classifier

In [10]:
microbe_ids = get_entities().loc[get_entities()['type'] == 'Microbe']['id'].tolist()
disease_ids = get_entities().loc[get_entities()['type'] == 'Disease']['id'].tolist()

In [11]:
mda_classifier = D3Classifier(microbe_ids, disease_ids)

2024-02-22 18:30:25,982 [INFO] Initializing AssessClassifier
2024-02-22 18:30:25,984 [INFO] Initializing MatrixFeatureExtractor


## Optimizer

In [12]:
classifier_optimizer_config = OptimizerConfig()
classifier_optimizer_config.exp_name = "assess optimizer"
classifier_optimizer_config.device = device
classifier_optimizer_config.threshold = 0.5

## Train Test Approach

### Train

In [13]:
train_result = DiseaseAssessClassifierTrainer().train(model=mda_classifier,
                                                          data=train_data,
                                                          config=classifier_optimizer_config)

2024-02-22 18:30:26,003 [INFO] Call Training with assess optimizer
2024-02-22 18:30:26,006 [INFO] Building D3Classifier
2024-02-22 18:30:26,013 [INFO] Calling build with associations :      disease  microbe  increased
876    13565    32044          0
542    43372    59219          0
596    31069    33211          0
432    12403    11709          1
786    13213    61523          0
..       ...      ...        ...
468    50436    64598          0
795     4928    57280          0
577      200    15528          0
843    31069    55403          0
132     3127    29241          1

[628 rows x 3 columns]
2024-02-22 18:30:26,150 [INFO] interaction matrix with shape (5179, 5645) has built
2024-02-22 18:30:26,438 [INFO] mask matrix with shape (5179, 5645) has built. This matrix shows not non elements.
2024-02-22 18:30:28,286 [INFO] interaction has been imputed to delete nans
2024-02-22 18:30:28,324 [INFO] d3 : [43621, 33293, 12403]
2024-02-22 18:30:28,348 [INFO] Result on Train Data : {'AUC': 0.

### Test

In [14]:
test_result = DiseaseAssessClassifierTester().test(model=mda_classifier,
                                                       data=test_data,
                                                       config=classifier_optimizer_config)

2024-02-22 18:30:28,373 [INFO] Call Testing with assess optimizer
2024-02-22 18:30:28,397 [INFO] Result on Test Data : {'AUC': 0.8152454780361758, 'ACC': 0.6703703703703704, 'F1 Score': 0.6640617354713341, 'AUPR': 0, 'Loss': 0}


In [15]:
test_result.get_result()

{'AUC': 0.8152454780361758,
 'ACC': 0.6703703703703704,
 'F1 Score': 0.6640617354713341,
 'AUPR': 0,
 'Loss': 0}

## Cross Validation

In [16]:
trainer = DiseaseAssessClassifierTrainer()
tester = DiseaseAssessClassifierTester()
factory = D3ClassifierFactory(microbe_ids, disease_ids)
spliter = MicrobeDiseaseAssociationTrainTestSpliter(data.associations)
cross_validation(k=5, data_size=data.associations.shape[0], train_test_spliter=spliter, model_factory=factory,
                    trainer=trainer, tester=tester, config=classifier_optimizer_config)

2024-02-22 18:30:28,442 [INFO] Initializing AssessClassifierFactory
2024-02-22 18:30:28,444 [INFO] Initializing MicrobeDiseaseAssociationTrainTestSpliter
2024-02-22 18:30:28,447 [INFO] Start 5-fold Cross Validation with config : assess optimizer
2024-02-22 18:30:28,451 [INFO] ---- Fold 1 ----
2024-02-22 18:30:28,455 [INFO] Initializing AssessClassifier
2024-02-22 18:30:28,457 [INFO] Initializing MatrixFeatureExtractor
2024-02-22 18:30:28,459 [INFO] Call Training with assess optimizer
2024-02-22 18:30:28,460 [INFO] Building D3Classifier
2024-02-22 18:30:28,466 [INFO] Calling build with associations :      disease  microbe  increased
0      50863    33211          1
1      43621    40832          1
2      33293    47880          1
4      33293    14909          1
5      33293    35937          1
..       ...      ...        ...
888    44991    20358          0
889      200    32631          0
892    22068    20153          0
894    25026    60601          0
897    64642     4251         